In [ ]:
! pip install langchain sentence-transformers faiss-gpu pypdf langchain_ollama 

In [ ]:
! pip install -U langchain-community 

In [ ]:
! pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama


In [ ]:
loader = PyPDFLoader("/content/got.pdf")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")

docs = text_splitter.split_documents(documents=documents)

In [ ]:
# Loading the embedding model from huggingface

embedding_model = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs=model_kwargs
)

In [ ]:
"""
vector_store = FAISS.from_documents(text_chunks, embeddings)
retriever = vector_store.as_retriever()

"""

In [ ]:
# Loading data and correspond embedding into FAISS
vector_store = FAISS.from_documents(documents=docs, embedding=embeddings)

In [ ]:
# Persist the vector locally on disk
vector_store.save_local("faiss_index_")

In [ ]:
# Load from local sotrage
persisted_vector_store = FAISS.load_local("faiss_index_", embeddings=embeddings, allow_dangerous_deserialization=True)

In [ ]:
# Create a retriever on top of database
retriever = persisted_vector_store.as_retriever()

In [ ]:
# Initialize an instance of the Ollama model
llm = Ollama(model="llama3.1")

In [ ]:
# Invoke the model to generate responses 
response = llm.invoke("Tell me about AI")
print(response)

In [ ]:
"""
from langchain_ollama.llms import OllamaLLM
# Loading Ollama model
model = OllamaLLM(model='llama3.1')

"""

In [ ]:
# Use RetrivelQA chain for orchestration
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
while True:
    query = input("Type your query if you want to exit type Exit: \n")
    query = query.lower()

    if query == "Exit":
        break

    result_1 = qa.run(query)
    print(result_1)

In [ ]:
# Some queries for testing 
query_1 = "Describe the relationship and dynamic between Will, Gared and Ser Waymar Royce"
query_2 = "How long have Gared and Will been part of the Night's watch?"